## CUDA

In [ ]:
import sys
print(sys.executable)
!{sys.executable} --version

In [2]:
!/usr/local/cuda-12.4/bin/nvcc --version
print("Nvidia drivers: ", end="")
!nvidia-smi --query-gpu=driver_version --format=csv,noheader

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Nvidia drivers: 550.120


## Install PyTorch

In [ ]:
# https://pytorch.org/get-started/locally/
!{sys.executable} -m pip install torch

In [4]:
import torch

print("torch: ", torch.__version__)
print(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

torch:  2.5.1+cu124
cuda


## Install PyTorch Geometric

In [ ]:
# https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html
!{sys.executable} -m pip install torch_geometric

In [6]:
import torch_geometric

print("torch_geometric: ", torch_geometric.__version__)

torch_geometric:  2.6.1


In [7]:
from torch_geometric.data import Data
import torch

# Create a small graph and send it to the GPU
edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
data = data.to('cuda')

print(data.x.is_cuda)

True


## Test installation

In [9]:
from torch_geometric.data import Data
import torch_geometric.nn as pyg_nn

# Create a simple graph data example
edge_index = torch.tensor([[0, 1, 1], [1, 0, 2]], dtype=torch.long)
x = torch.tensor([[1, 2], [2, 3], [3, 4]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Move data to the specified device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
print("Data device:", data.x.device)

# Define a simple GCN model
class SimpleGNN(torch.nn.Module):
    def __init__(self):
        super(SimpleGNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(2, 2)
        self.conv2 = pyg_nn.GCNConv(2, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize and move the model to the device
model = SimpleGNN().to(device)
print("Model device:", next(model.parameters()).device)

# Perform a forward pass
output = model(data)
print("Model output:", output)

Data device: cuda:0
Model device: cuda:0
Model output: tensor([[-0.1295,  0.0694],
        [-0.1295,  0.0694],
        [-0.0988,  0.0530]], device='cuda:0', grad_fn=<AddBackward0>)
